# How to use the FIM-MJP Model 

## Input to the model

The model takes as an input dictionary containing at least three items and one additional argument. The input dictionary should contain the following items:

   1. The _observation grid_ with size `[num_paths, grid_size]` which are the locations in time when a observation was recorded. The key in the dictionary is `observation_grid` and the data type is `float`.
   2. The _observation values_ with size `[num_paths, grid_size]` are the actually observed values (state) of the process. The key in the dictionary is `observation_values` and the data type is `int`.
   3. The _sequence length_ with size `[num_paths]` which is the length of the observed sequence. The key in the dictionary is `seq_length` and the data type is `int`.
   4. The _dimension of the process_ which is an `integer` between 2 and 6. The maximum number of states that are supported by our model is 6. The argument name is `n_states`.

Optionally, the dictionary can contain the following items:

   - The _time normalization factor_ with size `[num_paths]` which is the factor by which the time is normalized. The key in the dictionary is `time_normalization_factors` and the data type is `float`. In case this item is not provided, the model will normalize the time by the maximum time in the observation grid.
   - Items for calculating the loss:
      - _intensity matrix_ with size `[num_paths, n_states, n_states]` which is the intensity matrix of the process. The key in the dictionary is `intensity_matrices` and the data type is `float`.
      - _initial distribution_ with size `[num_paths, n_states]` which is the initial distribution of the process. The key in the dictionary is `initial_distributions` and the data type is `int`.
      - _adjacency matrix_ with size `[num_paths, n_states, n_states]` which is the adjacency matrix of the process. The key in the dictionary is `adjacency_matrices` and the data type is `int`.

### Output of the model

The model returns a dictionary containing the following items:

   - The _intensity matrix_ with size `[num_paths, n_states, n_states]` which is the intensity matrix of the process. The key in the dictionary is `intensity_matrices` and the data type is `float`.
   - The _initial distribution_ with size `[num_paths, n_states]` which is the initial distribution of the process. The key in the dictionary is `initial_distributions` and the data type is `int`.
   - The _adjacency matrix_ with size `[num_paths, n_states, n_states]` which is the adjacency matrix of the process. The key in the dictionary is `adjacency_matrices` and the data type is `int`.
   - The _losses_ which is the loss of the model. The key in the dictionary is `loss` and the data type is `float`.



## Loading the data and our model

In [9]:
from datasets import load_dataset
from transformers import AutoModel
import torch
from collections import defaultdict
from fim.trainers.utils import get_accel_type
import pandas as pd

device = get_accel_type()

## Dataset

We also provide a synthetic dataset for evaluating the model

In [10]:
# Loading the Discrete Flashing Ratchet (DFR) dataset from Huggingface
data = load_dataset("FIM4Science/mjp", download_mode="force_redownload", trust_remote_code=True, name="DFR_V=1")
data.set_format("torch")

mjp.py:   0%|          | 0.00/7.00k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


6_st_DFR_V=1.zip:   0%|          | 0.00/253k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

## Pretrained model

In [11]:
# Loading the FIMMJP model from Huggingface
fimmjp = AutoModel.from_pretrained("FIM4Science/fim-mjp", trust_remote_code=True)

fimmjp = fimmjp.to(device)
fimmjp.eval()

FIMMJP(
  (gaussian_nll): GaussianNLLLoss()
  (init_cross_entropy): CrossEntropyLoss()
  (pos_encodings): DeltaTimeEncoding()
  (ts_encoder): RNNEncoder(
    (rnn): LSTM(8, 256, batch_first=True, bidirectional=True)
  )
  (path_attention): MultiHeadLearnableQueryAttention(
    (W_k): Linear(in_features=512, out_features=128, bias=False)
    (W_v): Linear(in_features=512, out_features=128, bias=False)
  )
  (intensity_matrix_decoder): MLP(
    (layers): Sequential(
      (linear_0): Linear(in_features=2049, out_features=128, bias=True)
      (activation_0): SELU()
      (linear_1): Linear(in_features=128, out_features=128, bias=True)
      (activation_1): SELU()
      (output): Linear(in_features=128, out_features=60, bias=True)
    )
  )
  (initial_distribution_decoder): MLP(
    (layers): Sequential(
      (linear_0): Linear(in_features=2049, out_features=128, bias=True)
      (activation_0): SELU()
      (linear_1): Linear(in_features=128, out_features=128, bias=True)
      (activati

In [12]:
# copy data to device
batch = {k: v.to(device) for k, v in data["train"][:1].items()}

In [13]:
# Prepare a batch
n_paths_eval = [1, 30, 100, 300, 500, 1000, 5000]
total_n_paths = batch["observation_grid"].shape[1]
statistics = total_n_paths // 300 

## Evaluate the model

In [14]:
result = defaultdict(list)
intensity_matrices=[]
with torch.no_grad():
    for n_paths in n_paths_eval:
        for _ in range(statistics):
            paths_idx = torch.randperm(total_n_paths)[:n_paths]
            mini_batch = batch.copy()
            mini_batch["observation_grid"] = batch["observation_grid"][:, paths_idx]
            mini_batch["observation_values"] = batch["observation_values"][:, paths_idx]
            mini_batch["seq_lengths"] = batch["seq_lengths"][:, paths_idx]
            output = fimmjp(mini_batch, n_states=6)
            intensity_matrices.append(output["intensity_matrices"])
            result[n_paths].append(output["losses"]["rmse_loss"].item())

In [15]:
mean_rates=torch.mean(torch.stack(intensity_matrices,dim=0),dim=0)[0].numpy() # Get the average predicted intensity matrix
mean_rates

array([[-1.8726817 ,  0.51337844,  0.34576902,  0.79274243,  0.10118372,
         0.11960831],
       [ 1.3319315 , -3.0184882 ,  0.6533575 ,  0.14318156,  0.756899  ,
         0.13311861],
       [ 2.1708746 ,  1.3002527 , -4.6813445 ,  0.28341696,  0.217854  ,
         0.7089467 ],
       [ 0.7624738 ,  0.13706328,  0.15247008, -2.6725833 ,  0.8516975 ,
         0.76887864],
       [ 0.1258811 ,  0.7281811 ,  0.12992634,  0.7655206 , -2.5777812 ,
         0.82827175],
       [ 0.13955928,  0.12330879,  0.767207  ,  0.73767596,  0.719442  ,
        -2.4871929 ]], dtype=float32)

In [ ]:
means = {n_paths: torch.tensor(losses).mean().item() for n_paths, losses in result.items()}
stds = {n_paths: torch.tensor(losses).std().item() for n_paths, losses in result.items()}

df_result = pd.DataFrame(
    {
        "# Paths during Evaluation": list(means.keys()),
        "RMSE": [f"{mean:.3f} ± {std:.3f}" for mean, std in zip(means.values(), stds.values())],
    }
)

df_result

,# Paths during Evaluation,RMSE
0,1,0.662 ± 0.049
1,30,0.285 ± 0.066
2,100,0.172 ± 0.028
3,300,0.119 ± 0.018
4,500,0.115 ± 0.017
5,1000,0.235 ± 0.019
6,5000,0.882 ± 0.001


: 